In [3]:
import os
import re #regex as re
import numpy as np
import pandas as pd

In [9]:
blob = dict()
folders = ["logs_complex_baseline"]

keys = ["tag", 
        "solver", 
        "nprocs", 
        "nthreads", 
        "mesh_npts", 
        "ncomponents"]
measures_teuchos = ["Assemble Time",
                    "BEM Reinitialisation Time", 
                    "Epetra_CrsMatrix::Multiply(TransA,X,Y)",
                    "LAC Solve Time",
                    "Local Evaluation Time",
                    "Multipole Generation Time",
                    "Multipole Integral Time",
                    "Multipole MatrVec Products Time",
                    "Solve Time"
                   ]
measures_perf = ["task-clock",
                 "cycles",
                 "instructions",
                 "branches",
                 "branch-misses",
                 "L1-dcache-loads",
                 "L1-dcache-load-misses",
                 "LLC-loads",
                 "LLC-load-misses",
                 "time elapsed",
                 "user",
                 "sys",
                ]
cols = keys + measures_teuchos + measures_perf

In [10]:
def parse_filename(fname):
    ret = [#"", #"tag", 
           "", #"solver", 
           1, #"nprocs", 
           1, #"nthreads", 
           #"", #"mesh_npts", 
           1, #"ncomponents"
          ]
    
    if "direct" in fname:
        ret[0] = "direct"
    else:
        ret[0] = "fma"
    
    m = re.search(r"_np(\d+)", fname)
    if m:
        ret[1] = int(m.group(1))
    m = re.search(r"_nt(\d+)", fname)
    if m:
        ret[2] = int(m.group(1))
    
    if "complex" in fname:
        ret[3] = 2
        
    return ret

def load_file(path, blob, tag, solver, nprocs, nthreads, ncomponents):
    with open(path, "r") as f:
        record = dict()
        record["tag"] = tag
        record["solver"] = solver
        record["nprocs"] = nprocs
        record["nthreads"] = nthreads
        record["ncomponents"] = ncomponents
        
        reading = False
        measures_teuchos_remaining = set(measures_teuchos)
        measures_perf_remaining = set(measures_perf)
        for l in f.readlines():
            tokens = [t for t in l[:-1].split(" ") if t]

            if l.startswith("   Number of degrees of freedom: "):
                record["mesh_npts"] = int(tokens[-1])
            elif l.startswith("Timer Name"):
                #new record; reset previous?
                
                reading = True
            elif reading:
                found = False
                #try to get teuchos measures
                for s in measures_teuchos_remaining:
                    if l.startswith(s):
                        #discard the call count, there remains the time in s
                        tmp = float(tokens[-2])
                        if record["nprocs"] > 1:
                            tmp = float(tokens[-4])
                        #print(f"found {s} in line; attempting to add {tmp}")
                        #print(f"tokens are {tokens}")
                        record[s] = float(tmp)
                        
                        measures_teuchos_remaining.remove(s)
                        found = True
                        break
                
                if not found:
                    for s in measures_perf_remaining:
                        if s in l:
                            #the interesting value is in tokens[0], but it usually requires stripping "," or UoM conversion
                            tmp = tokens[0].replace(",", "")
                            #print(f"found {s} in line; attempting to add {tmp}")
                            #print(f"tokens are {tokens}")
                            
                            if s == "task-clock":
                                record[s] = float(tmp) / 1000
                            elif "." in tmp:
                                record[s] = float(tmp)
                            else:
                                record[s] = int(tmp)
                                
                            measures_perf_remaining.remove(s)
                            found = True
                            break
                #print(f"remaining keys: {len(measures_teuchos_remaining)} and {len(measures_perf_remaining)}")
                if not measures_teuchos_remaining and not measures_perf_remaining:
                    #finished a record
                    #print(f"finished a new record")
                    for k, v in record.items():
                        #print(f"{k}\t\t{v}")
                        blob[k].append(v)
                        
                    reading = False
                    measures_teuchos_remaining = set(measures_teuchos)
                    measures_perf_remaining = set(measures_perf)

In [12]:
#empty blob on which to build, once filled, the dataframe
blob = dict()
for c in cols:
    blob[c] = list()

revisions = ["baseline",
             #"tightening"
            ]
for r in revisions:
    dname = os.path.join(os.getcwd(), "logs_" + r)
    for fname in os.listdir(dname):

        load_file(os.path.join(dname, fname),
                  blob, r, *parse_filename(fname))

#print(blob)
df = pd.DataFrame.from_dict(blob)
#df.head()
df.groupby(keys).mean()

Assemble Time  \
tag      solver nprocs nthreads mesh_npts ncomponents                  
baseline direct 1      1        8995      1                154.52500   
                                          2                152.77500   
                2      1        8995      1                 84.98625   
                                          2                 83.49500   
                4      1        8995      1                 49.21750   
                                          2                 48.93750   
         fma    1      1        8995      1                  0.00000   
                                          2                  0.00000   
                2      1        8995      1                  0.00000   
                                          2                  0.00000   
                4      1        8995      1                  0.00000   
                                          2                  0.00000   

                                                       BEM Reinitialisation Time  \
tag      solver nprocs nthreads mesh_npts ncomponents                              
baseline direct 1      1        8995      1                             4.502583   
                                          2                             4.466500   
                2      1        8995      1                             3.291375   
                                          2                             3.315750   
                4      1        8995      1                             2.299750   
                                          2                             2.324500   
         fma    1      1        8995      1                             0.071781   
                                          2                             0.071527   
                2      1        8995      1                             0.221774   
                                          2                             0.241625   
                4      1        8995      1                             0.213313   
                                          2                             0.397725   

                                                       Epetra_CrsMatrix::Multiply(TransA,X,Y)  \
tag      solver nprocs nthreads mesh_npts ncomponents                                           
baseline direct 1      1        8995      1                                         13.196667   
                                          2                                         25.970000   
                2      1        8995      1                                          7.946625   
                                          2                                         15.782500   
                4      1        8995      1                                          6.743000   
                                          2                                         13.432500   
         fma    1      1        8995      1                                          0.304867   
                                          2                                          0.601525   
                2      1        8995      1                                          0.353500   
                                          2                                          0.697750   
                4      1        8995      1                                          0.425762   
                                          2                                          0.837375   

                                                       LAC Solve Time  \
tag      solver nprocs nthreads mesh_npts ncomponents                   
baseline direct 1      1        8995      1                 14.455000   
                                          2                 28.365000   
                2      1        8995      1                 12.418750   
                                          2                 19.725000   
                4      1        8995      1                  9.396375   